In [4]:
import cv2
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [5]:
from PIL import Image, ImageEnhance

In [13]:
image='pizza_1.jpg'

In [7]:
# first to preprocess the image to make it to matrix of integers
def preprocess_image(img_path):
  img=Image.open(img_path)
  enhancer=ImageEnhance.Brightness(img)
  brightened_img=enhancer.enhance(1.5)

  #resize to 4% of the image
  width, height=brightened_img.size
  new_size=(int(width*0.04), int(height*0.04))
  resized_img=brightened_img.resize(new_size)

  return resized_img

In [8]:
# to extract RGB channels as features to analyse
def extract_rgb(img):
  np_img=np.array(img)
  red_channel=np_img[:,:,0].flatten()
  green_channel=np_img[:,:,1].flatten()
  blue_channel=np_img[:,:,2].flatten()
  # to make one feature vector
  return np.concatenate([red_channel, green_channel, blue_channel])


In [9]:
# to try other methods like average of RGB and GRAYSCALE
def grayscale_avg_rgb(img):
  np_img=np.array(img)
  gray_image_cvt=cv2.cvtColor(np_img, cv2.COLOR_BGR2GRAY)
  avg_rgb=np.mean(np_img, axis=2).flatten()
  return gray_image_cvt.flatten(), avg_rgb

In [10]:
def main(img_path):
  img=preprocess_image(img_path)
  rgb_features=extract_rgb(img)
  print(f"RGB Features: {len(rgb_features)} features extracted")
  gray_features, avg_rgb_features=grayscale_avg_rgb(img)
  print(f"Grayscale Features: {len(gray_features)} features extracted")
  print(f"Average RGB Features: {len(avg_rgb_features)} features extracted")

In [15]:
main(image)

RGB Features: 7200 features extracted
Grayscale Features: 2400 features extracted
Average RGB Features: 2400 features extracted


In [16]:
def split_data(X,y):
  X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

In [17]:
def feature_selection_infogain(X_train, y_train, num_features=3000):
  selector=SelectKBest(mutual_info_classif, k=num_features)
  X_train_selected=selector.fit_transform(X_train, y_train)
  return X_train_selected, selector

In [18]:
def apply_pca(X_train_selected):
  pca=PCA(n_components=0.95)
  X_train_pca=pca.fit_transform(X_train_selected)
  return X_train_pca, pca

In [20]:
#food size classification
def food_size_classification(X_train_pca, y_train):
  model=RandomForestClassifier()
  model.fit(X_train_pca, y_train)
  return model

In [ ]:
#food type classification
from sklearn
def food_type_classification(X_train_pca, y_train):
  model=